In [4]:
import yfinance as yf
import pandas as pd

In [71]:
def get_stock_data(tickers, start, end):
    data = {}

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        df = stock.history(start=start, end=end)

        # Get the income statement (Net Income)
        income_statement = stock.quarterly_financials.T  # Transpose to get data as rows
        if 'Net Income' in income_statement.columns:
            net_income = income_statement['Net Income']
            net_income = net_income.reset_index()  # Make sure the index is datetime
            net_income.columns = ['Date', 'Net Income']
        else:
            print(f"No Net Income data for {ticker}")
            continue  # Skip if no income data is available

        # Convert Date columns to timezone-naive to match the stock data
        net_income['Date'] = pd.to_datetime(net_income['Date']).dt.tz_localize(None)
        df = df.reset_index()
        df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)

        # Merge EPS with stock prices (assuming you calculate EPS as shown earlier)
        df = pd.merge_asof(df, net_income[['Date', 'Net Income']], on='Date', direction='backward')

        # Calculate P/E Ratio
        df['P/E'] = df['Close'] / df['Net Income']  # You may need to adjust this if using EPS

        # Calculate P/S Ratio (using total revenue)
        if 'Total Revenue' in income_statement.columns:
            total_revenue = income_statement['Total Revenue']
            total_revenue = total_revenue.reset_index()
            total_revenue.columns = ['Date', 'Total Revenue']
            total_revenue['Date'] = pd.to_datetime(total_revenue['Date']).dt.tz_localize(None)
            df = pd.merge_asof(df, total_revenue[['Date', 'Total Revenue']], on='Date', direction='backward')
            df['P/S'] = df['Close'] / df['Total Revenue']
        else:
            df['P/S'] = None  # Mark as None if not available

        data[ticker] = df

    return data

In [72]:
dotcom_stocks = ['AMZN', 'CSCO', 'INTC']
ai_stocks = ['GOOG', 'MSFT', 'NVDA']

dotcom_start = '1996-01-01'
dotcom_end = '2001-01-01'

ai_start = '2020-01-01'
ai_end = '2024-10-10'

In [73]:
dotcom_data = get_stock_data(dotcom_stocks, dotcom_start, dotcom_end)

ValueError: right keys must be sorted

In [58]:
ai_data = get_stock_data(ai_stocks, ai_start, ai_end)

dict_keys(['address1', 'city', 'state', 'zip', 'country', 'phone', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'compensationAsOfEpochDate', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'beta', 'trailingPE', 'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currency', 'enterpriseValue', 'profitMargins', 'floatShares', 'sharesOutstanding', 'sharesShort', 'sharesShortPriorMonth', 'sharesShortPreviousMonthDate', 'd

In [59]:
df_ai = pd.concat(ai_data.values(), keys=ai_data.keys(), names=['Ticker', 'Date'])

In [60]:
df_dotcom = pd.concat(dotcom_data.values(), keys=dotcom_data.keys(), names=['Ticker', 'Date'])

In [61]:
df_ai.head(50)

Open       High        Low      Close  \
Ticker Date                                                                    
GOOG   2020-01-02 00:00:00-05:00  66.913174  68.239415  66.913174  68.201012   
       2020-01-03 00:00:00-05:00  67.227894  68.456878  67.112379  67.866325   
       2020-01-06 00:00:00-05:00  67.334639  69.653940  67.334639  69.539726   
       2020-01-07 00:00:00-05:00  69.725770  69.977646  69.348690  69.496330   
       2020-01-08 00:00:00-05:00  69.433483  70.406101  69.371639  70.043991   
       2020-01-09 00:00:00-05:00  70.854493  71.191662  70.340748  70.817581   
       2020-01-10 00:00:00-05:00  71.203139  71.570690  70.743772  71.311378   
       2020-01-13 00:00:00-05:00  71.630591  71.849551  71.126328  71.785210   
       2020-01-14 00:00:00-05:00  71.774235  71.913389  71.243540  71.368729   
       2020-01-15 00:00:00-05:00  71.335307  71.893187  71.335307  71.783707   
       2020-01-16 00:00:00-05:00  72.194705  72.421649  71.869500  72.407181   
       2020-01-17 00:00:00-05:00  72.966306  73.883304  72.732383  73.838165   
       2020-01-21 00:00:00-05:00  73.774824  74.409763  73.379791  74.038177   
       2020-01-22 00:00:00-05:00  74.367370  74.976569  74.064609  74.115486   
       2020-01-23 00:00:00-05:00  74.199784  74.592816  73.923462  74.150398   
       2020-01-24 00:00:00-05:00  74.496547  74.591567  73.083019  73.155846   
       2020-01-27 00:00:00-05:00  71.374722  71.727356  70.885917  71.519363   
       2020-01-28 00:00:00-05:00  71.973255  72.621664  71.448040  72.450081   
       2020-01-29 00:00:00-05:00  72.761309  73.091994  72.159782  72.752831   
       2020-01-30 00:00:00-05:00  71.821617  72.685493  71.644051  72.613670   
       2020-01-31 00:00:00-05:00  73.265071  73.326421  71.251515  71.535820   
       2020-02-03 00:00:00-05:00  72.920916  74.317488  72.770788  74.114983   
       2020-02-04 00:00:00-05:00  72.675022  73.294997  71.140293  72.176247   
       2020-02-05 00:00:00-05:00  72.941873  73.012698  71.352773  72.234108   
       2020-02-06 00:00:00-05:00  72.338851  73.918467  72.300942  73.630676   
       2020-02-07 00:00:00-05:00  73.185269  74.110000  73.137884  73.780312   
       2020-02-10 00:00:00-05:00  73.535420  75.290106  73.535420  75.249207   
       2020-02-11 00:00:00-05:00  75.405323  76.294139  75.097478  75.254692   
       2020-02-12 00:00:00-05:00  75.538489  75.848481  75.220773  75.727524   
       2020-02-13 00:00:00-05:00  75.449208  76.171936  75.045704  75.547470   
       2020-02-14 00:00:00-05:00  75.594357  75.850731  75.182366  75.850731   
       2020-02-18 00:00:00-05:00  75.564430  76.393890  75.444227  75.797356   
       2020-02-19 00:00:00-05:00  76.066696  76.417633  75.883644  76.147499   
       2020-02-20 00:00:00-05:00  75.913568  76.294636  75.156432  75.721542   
       2020-02-21 00:00:00-05:00  75.216774  75.425513  73.840660  74.073586   
       2020-02-24 00:00:00-05:00  71.130809  71.672486  70.396620  70.905365   
       2020-02-25 00:00:00-05:00  71.474470  71.730837  68.950669  69.252426   
       2020-02-26 00:00:00-05:00  69.635990  70.611599  68.781087  69.488350   
       2020-02-27 00:00:00-05:00  67.936158  68.417177  65.697157  65.743050   
       2020-02-28 00:00:00-05:00  63.718519  66.892723  63.394314  66.802444   
       2020-03-02 00:00:00-05:00  67.414940  69.373136  66.178231  69.285347   
       2020-03-03 00:00:00-05:00  69.799586  70.334771  66.436842  66.905197   
       2020-03-04 00:00:00-05:00  67.795014  69.234480  66.990989  69.156166   
       2020-03-05 00:00:00-05:00  67.344618  67.779052  65.095137  65.790436   
       2020-03-06 00:00:00-05:00  63.696573  65.150998  62.898036  64.761459   
       2020-03-09 00:00:00-04:00  60.117361  62.584302  59.853011  60.629105   
       2020-03-10 00:00:00-04:00  62.845660  63.900572  60.789210  63.862663   
       2020-03-11 00:00:00-04:00  62.331924  62.893545  59.656994  60.621624   
       2020-03-12 00:00:00-04:00  56.16

In [36]:
df_dotcom.head()

Open      High       Low     Close  \
Ticker Date                                                                
AMZN   1997-05-15 00:00:00-04:00  0.121875  0.125000  0.096354  0.097917   
       1997-05-16 00:00:00-04:00  0.098438  0.098958  0.085417  0.086458   
       1997-05-19 00:00:00-04:00  0.088021  0.088542  0.081250  0.085417   
       1997-05-20 00:00:00-04:00  0.086458  0.087500  0.081771  0.081771   
       1997-05-21 00:00:00-04:00  0.081771  0.082292  0.068750  0.071354   

                                      Volume  Dividends  Stock Splits  \
Ticker Date                                                             
AMZN   1997-05-15 00:00:00-04:00  1443120000        0.0           0.0   
       1997-05-16 00:00:00-04:00   294000000        0.0           0.0   
       1997-05-19 00:00:00-04:00   122136000        0.0           0.0   
       1997-05-20 00:00:00-04:00   109344000        0.0           0.0   
       1997-05-21 00:00:00-04:00   377064000        0.0           0.0   

                                       P/E       P/S  
Ticker Date                                           
AMZN   1997-05-15 00:00:00-04:00  45.18424  3.280148  
       1997-05-16 00:00:00-04:00  45.18424  3.280148  
       1997-05-19 00:00:00-04:00  45.18424  3.280148  
       1997-05-20 00:00:00-04:00  45.18424  3.280148  
       1997-05-21 00:00:00-04:00  45.18424  3.280148

In [37]:
# save to csv 
df_ai.to_csv('ai_stocks.csv')
df_dotcom.to_csv('dotcom_stocks.csv')